# Best practice for plotting batch-runs

In [ ]:
import os
import bokeh
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cellpy import cellreader
from cellpy.utils import ica
import holoviews as hv
import altair as alt
%matplotlib inline

### Setting file names and loading data

In [ ]:
my_data = cellreader.CellpyData()
filename = "../testdata/hdf5/20160805_test001_45_cc.h5"
assert os.path.isfile(filename)

In [ ]:
my_data.load(filename)
my_data.set_mass(0.1)

### Checking the data

#### voltage cycles

In [ ]:
capacity, voltage = my_data.get_cap()

In [ ]:
plt.plot(capacity, voltage)

In [ ]:
cycles = my_data.get_cycle_numbers()

In [ ]:
charge_cycles = np.array([my_data.get_ccap(c) for c in cycles])

In [ ]:
for x,y in charge_cycles:
    plt.plot(x,y)

In [ ]:
def make_charge_list(data):
    """Create a list of pandas.DataFrames, one for each charge step.
    
    The DataFrames are named by its cycle number.
    
    Input: CellpyData
    Returns: list of pandas.DataFrames
        minimum voltage value,
        maximum voltage value"""
    
    minimum_v_value = np.Inf
    maximum_v_value = -np.Inf
    charge_list = []
    cycles = my_data.get_cycle_numbers()
    for cycle in cycles:
        q, v = data.get_ccap(cycle)
        d = pd.DataFrame({"q": q, "v": v})
        #d.name = f"{cycle}"
        d.name = cycle
        charge_list.append(d)
        v_min = v.min()
        v_max = v.max()
        if v_min < minimum_v_value:
            minimum_v_value = v_min
        if v_max > maximum_v_value:
            maximum_v_value = v_max
    return charge_list, minimum_v_value, maximum_v_value


In [ ]:
charge_dfs, minimum_v, maximum_v = make_charge_list(my_data)

In [ ]:
charge_df = pd.concat(charge_dfs, axis=1, keys=[k.name for k in charge_dfs])

In [ ]:
# discharge_cycles = np.array([my_data.get_dcap(c) for c in cycles])

### Checking ica

In [ ]:
c, v = charge_cycles[2]
dq, dv = ica.dqdv(v, c)
plt.plot(dq,dv)

In [ ]:
print(f"min, max = ({minimum_v} {maximum_v})")

## Creating dqdv-data

In [ ]:
def custom_dq_dv(capacity, voltage, minimum_v, maximum_v):
    converter = ica.Converter()
    converter.set_data(capacity, voltage)
    converter.inspect_data()
    converter.pre_process_data()
    converter.increment_data()
    converter.fixed_voltage_range = [minimum_v, maximum_v, 100]
    converter.post_process_data()
    return converter.voltage_processed, converter.incremental_capacity

In [ ]:
incremental_charge_list = []
for (cap, volt), cycle in zip(charge_cycles, cycles):
    if cap.any():
        dv, dq = custom_dq_dv(cap, volt, minimum_v, maximum_v)
        if not incremental_charge_list:
            d = pd.DataFrame({"dv": dv})
            d.name = "voltage"
            incremental_charge_list.append(d)

            d = pd.DataFrame({f"dq": dq})
            d.name = cycle
            incremental_charge_list.append(d)
            
        else:
            d = pd.DataFrame({f"dq": dq})
            #d.name = f"{cycle}"
            d.name = cycle
            incremental_charge_list.append(d)
    else:
        print(f"{cycle} is empty")

In [ ]:
ica_df = pd.concat(incremental_charge_list, axis=1, keys=[k.name for k in incremental_charge_list])

In [ ]:
ica_df.columns.names = ["cycle", "value"]

In [ ]:
ica_df.head(12)

In [ ]:
ica_df.plot(x=("voltage","dv"))

**Success!**

In [ ]:
ica_df.head()

In [ ]:
ica_long_multiindex = ica_df.copy()

In [ ]:
ica_long_multiindex.head()

In [ ]:
long_multiindex = ica_long_multiindex.melt("voltage", var_name="cycle", value_name="dq", col_level=0)

In [ ]:
long_multiindex.head()

In [ ]:
long_multiindex.tail()

### Plotting with Altair

In [ ]:
alt.renderers.enable('notebook')

|   Data Type    |    Shorthand Code   |   Description                     |
|----------------|---------------------|-----------------------------------|
| quantitative   | Q                   | a continuous real-valued quantity |
|ordinal | O | a discrete ordered quantity|
|nominal | N | a discrete unordered category|
|temporal | T | a time or date value|

In [ ]:
alt.Chart(long_multiindex).mark_line().encode(
  x='voltage:Q',
  y='dq:Q',
  color='cycle:O'
).interactive()

In [ ]:
import altair as alt
from vega_datasets import data

stocks = long_multiindex

highlight = alt.selection(type='single', on='mouseover',
                          fields=['cycle'], nearest=True)

base = alt.Chart(stocks).encode(
    x='voltage:Q',
    y='dq:Q',
    color='cycle:O'
)

points = base.mark_circle().encode(
    opacity=alt.value(0)
).add_selection(
    highlight
).properties(
    width=600
)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(0.5), alt.value(3))
).interactive()

points + lines

Easy...but a bit limiting?

### Trying Bokeh

The main benifit with using Bokeh: interactive html

In [ ]:
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# trick to flatten the multiindex (not sure if HoloViews takes multiindex)
# ica_df.columns = ['_'.join(col).strip() for col in ica_df.columns.values]

In [ ]:
from bokeh.models import ColumnDataSource, DataRange1d, Plot, LinearAxis, Grid
from bokeh.plotting import figure, show
from bokeh.palettes import Spectral11, Paired, Viridis256
from bokeh.models.glyphs import MultiLine, Line
from bokeh.io import show
import itertools
from bokeh.transform import linear_cmap

In [ ]:
# source = ColumnDataSource(ica_df) # bokeh automatically flattens multiindexed columns using "_".join(k) (i.e. must be str)
# source.data

In [ ]:
source = ColumnDataSource(long_multiindex)

In [ ]:
xrange = DataRange1d()
yrange = DataRange1d()
plot = figure(x_range=xrange, y_range=yrange, plot_width=800, plot_height=300)
plot.line(source=source, x="voltage", y="dq", legend="cycle") #, line_color=linear_cmap('cycle', "Viridis256", 0, 255))
show(plot)
# How can I set a color palette? mappers?

Not sure if I want to do this anymore. Bokeh is a little bit of a "luremus"..
Lets check if Holoviews makes life a little bit easier.

### Plotting with HoloViews

In [ ]:
hv.extension('bokeh')

In [ ]:
curve = hv.Curve(long_multiindex, kdims=['voltage', "dq"], vdims=['cycle'])
curve

In [ ]:
%%opts
curve2 = hv.Curve(long_multiindex, kdims=["voltage", "dq"], vdims="cycle")
curve2

In [ ]:
long_multiindex.head()

In [ ]:
from bokeh.sampledata.iris import flowers
from holoviews.operation import gridmatrix

iris_ds = hv.Dataset(flowers)

In [ ]:
%%opts Bivariate [bandwidth=0.5] (cmap='Blues') Points [tools=['box_select']] (size=2)
density_grid = gridmatrix(iris_ds, diagonal_type=hv.Distribution, chart_type=hv.Bivariate)
point_grid = gridmatrix(iris_ds, chart_type=hv.Points)

density_grid * point_grid